In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

In [76]:
df = pd.read_csv(r"C:\Users\iluha\Desktop\Air\The Table.csv")

In [77]:
df['satisfaction'] = df['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})

In [78]:
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 
                   'distance_category', 'age_group']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [79]:
cols_to_drop = ['id', 'Date', 'overall_service', 'comfort_score', 
                'entertainment_score', 'total_delay', 'has_delay', 
                'delay_ratio', 'low_ratings_count', 'high_ratings_count']
df = df.drop(columns=cols_to_drop, errors='ignore')

In [80]:
df = df.fillna(df.mean())
X = df.drop('satisfaction', axis=1)
y = df['satisfaction']

In [81]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [82]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [83]:
print(f"Размеры данных:")
print(f"Обучающая выборка: {X_train.shape}")
print(f"Валидационная выборка: {X_val.shape}")
print(f"Соотношение классов в обучающей выборке: {y_train.value_counts(normalize=True).to_dict()}")

Размеры данных:
Обучающая выборка: (83128, 29)
Валидационная выборка: (20782, 29)
Соотношение классов в обучающей выборке: {0: 0.5666682706188048, 1: 0.43333172938119524}


In [84]:
# Логистическая регрессия без сэмла
logreg_params = {
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'],
    'max_iter': [1000]
}

In [85]:
logreg = LogisticRegression(random_state=42)
grid_search_logreg = GridSearchCV(
    logreg, 
    logreg_params, 
    cv=5, 
    scoring='f1'
)

In [86]:
grid_search_logreg.fit(X_train_scaled, y_train)
best_logreg = grid_search_logreg.best_estimator_

In [87]:
print(f"Лучшие параметры: {grid_search_logreg.best_params_}")
print(f"Лучший F1 на кросс-валидации: {grid_search_logreg.best_score_:.4f}")

Лучшие параметры: {'C': 1, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}
Лучший F1 на кросс-валидации: 0.8535


In [88]:
y_pred = best_logreg.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

In [89]:
print(f"\nМетрики на валидационной выборке:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Метрики на валидационной выборке:
Accuracy: 0.8758
Precision: 0.8691
Recall: 0.8399
F1-score: 0.8542


In [90]:
# С сэмплом
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

In [91]:
print(f"Размер обучающей выборки после SMOTE: {X_train_smote.shape}")
print(f"Распределение классов после SMOTE: {pd.Series(y_train_smote).value_counts().to_dict()}")

Размер обучающей выборки после SMOTE: (94212, 29)
Распределение классов после SMOTE: {0: 47106, 1: 47106}


In [92]:
grid_search_logreg_smote = GridSearchCV(
    logreg, 
    logreg_params, 
    cv=5, 
    scoring='f1'
)

In [93]:
grid_search_logreg_smote.fit(X_train_smote, y_train_smote)
best_logreg_smote = grid_search_logreg_smote.best_estimator_

In [94]:
print(f"Лучшие параметры: {grid_search_logreg_smote.best_params_}")
print(f"Лучший F1 на кросс-валидации: {grid_search_logreg_smote.best_score_:.4f}")

Лучшие параметры: {'C': 0.01, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'saga'}
Лучший F1 на кросс-валидации: 0.8685


In [95]:
y_pred_smote = best_logreg_smote.predict(X_val_scaled)
accuracy_smote = accuracy_score(y_val, y_pred_smote)
precision_smote = precision_score(y_val, y_pred_smote)
recall_smote = recall_score(y_val, y_pred_smote)
f1_smote = f1_score(y_val, y_pred_smote)

In [96]:
print(f"\nМетрики на валидационной выборке:")
print(f"Accuracy: {accuracy_smote:.4f}")
print(f"Precision: {precision_smote:.4f}")
print(f"Recall: {recall_smote:.4f}")
print(f"F1-score: {f1_smote:.4f}")


Метрики на валидационной выборке:
Accuracy: 0.8682
Precision: 0.8408
Recall: 0.8583
F1-score: 0.8495


In [97]:
# KNN без сэмпла
knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

In [98]:
knn = KNeighborsClassifier()
grid_search_knn = GridSearchCV(
    knn, 
    knn_params, 
    cv=5, 
    scoring='f1'
)

In [99]:
grid_search_knn.fit(X_train_scaled, y_train)
best_knn = grid_search_knn.best_estimator_

In [100]:
print(f"Лучшие параметры: {grid_search_knn.best_params_}")
print(f"Лучший F1 на кросс-валидации: {grid_search_knn.best_score_:.4f}")

Лучшие параметры: {'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
Лучший F1 на кросс-валидации: 0.9141


In [101]:
y_pred_knn = best_knn.predict(X_val_scaled)

In [102]:
accuracy_knn = accuracy_score(y_val, y_pred_knn)
precision_knn = precision_score(y_val, y_pred_knn)
recall_knn = recall_score(y_val, y_pred_knn)
f1_knn = f1_score(y_val, y_pred_knn)

In [103]:
print(f"\nМетрики на валидационной выборке:")
print(f"Accuracy: {accuracy_knn:.4f}")
print(f"Precision: {precision_knn:.4f}")
print(f"Recall: {recall_knn:.4f}")
print(f"F1-score: {f1_knn:.4f}")


Метрики на валидационной выборке:
Accuracy: 0.9274
Precision: 0.9386
Recall: 0.8908
F1-score: 0.9141


In [104]:
# с сэмплом
grid_search_knn_smote = GridSearchCV(
    knn, 
    knn_params, 
    cv=5, 
    scoring='f1'
)

In [105]:
grid_search_knn_smote.fit(X_train_smote, y_train_smote)
best_knn_smote = grid_search_knn_smote.best_estimator_

In [106]:
print(f"Лучшие параметры: {grid_search_knn_smote.best_params_}")
print(f"Лучший F1 на кросс-валидации: {grid_search_knn_smote.best_score_:.4f}")

Лучшие параметры: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
Лучший F1 на кросс-валидации: 0.9396


In [107]:
y_pred_knn_smote = best_knn_smote.predict(X_val_scaled)

accuracy_knn_smote = accuracy_score(y_val, y_pred_knn_smote)
precision_knn_smote = precision_score(y_val, y_pred_knn_smote)
recall_knn_smote = recall_score(y_val, y_pred_knn_smote)
f1_knn_smote = f1_score(y_val, y_pred_knn_smote)

In [108]:
print(f"\nМетрики на валидационной выборке:")
print(f"Accuracy: {accuracy_knn_smote:.4f}")
print(f"Precision: {precision_knn_smote:.4f}")
print(f"Recall: {recall_knn_smote:.4f}")
print(f"F1-score: {f1_knn_smote:.4f}")


Метрики на валидационной выборке:
Accuracy: 0.9245
Precision: 0.9233
Recall: 0.9006
F1-score: 0.9118


In [109]:
# Decision Tree
tree_params = {
    'max_depth': [3, 5, 7, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['gini', 'entropy']
}

In [110]:
tree = DecisionTreeClassifier(random_state=42)
grid_search_tree = GridSearchCV(
    tree, 
    tree_params, 
    cv=5, 
    scoring='f1',
    n_jobs=-1,
    verbose=0
)

In [111]:
grid_search_tree.fit(X_train_scaled, y_train)
best_tree = grid_search_tree.best_estimator_

In [112]:
print(f"Лучшие параметры: {grid_search_tree.best_params_}")
print(f"Лучший F1 на кросс-валидации: {grid_search_tree.best_score_:.4f}")

Лучшие параметры: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 2, 'min_samples_split': 15}
Лучший F1 на кросс-валидации: 0.9417


In [113]:
y_pred_tree = best_tree.predict(X_val_scaled)

In [114]:
accuracy_tree = accuracy_score(y_val, y_pred_tree)
precision_tree = precision_score(y_val, y_pred_tree)
recall_tree = recall_score(y_val, y_pred_tree)
f1_tree = f1_score(y_val, y_pred_tree)

In [115]:
print(f"\nМетрики на валидационной выборке:")
print(f"Accuracy: {accuracy_tree:.4f}")
print(f"Precision: {precision_tree:.4f}")
print(f"Recall: {recall_tree:.4f}")
print(f"F1-score: {f1_tree:.4f}")


Метрики на валидационной выборке:
Accuracy: 0.9501
Precision: 0.9469
Recall: 0.9374
F1-score: 0.9421


In [116]:
# с сэмплом
grid_search_tree_smote = GridSearchCV(
    tree, 
    tree_params, 
    cv=5, 
    scoring='f1'
)

In [117]:
grid_search_tree_smote.fit(X_train_smote, y_train_smote)
best_tree_smote = grid_search_tree_smote.best_estimator_

In [118]:
print(f"Лучшие параметры: {grid_search_tree_smote.best_params_}")
print(f"Лучший F1 на кросс-валидации: {grid_search_tree_smote.best_score_:.4f}")

Лучшие параметры: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 15}
Лучший F1 на кросс-валидации: 0.9515


In [119]:
y_pred_tree_smote = best_tree_smote.predict(X_val_scaled)

In [120]:
accuracy_tree_smote = accuracy_score(y_val, y_pred_tree_smote)
precision_tree_smote = precision_score(y_val, y_pred_tree_smote)
recall_tree_smote = recall_score(y_val, y_pred_tree_smote)
f1_tree_smote = f1_score(y_val, y_pred_tree_smote)

In [121]:
print(f"\nМетрики на валидационной выборке:")
print(f"Accuracy: {accuracy_tree_smote:.4f}")
print(f"Precision: {precision_tree_smote:.4f}")
print(f"Recall: {recall_tree_smote:.4f}")
print(f"F1-score: {f1_tree_smote:.4f}")


Метрики на валидационной выборке:
Accuracy: 0.9488
Precision: 0.9429
Recall: 0.9387
F1-score: 0.9408


In [122]:
# Анализ
feature_importance = best_tree_smote.feature_importances_
feature_names = X.columns

In [123]:
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)


In [124]:
print("Топ-20 самых важных признаков:")
print(importance_df.head(20).to_string(index=False))

Топ-20 самых важных признаков:
                          Feature  Importance
                  Online boarding    0.352897
            Inflight wifi service    0.208404
   Type of Travel_Personal Travel    0.144500
     Customer Type_Loyal Customer    0.048244
           Inflight entertainment    0.043725
                 On-board service    0.037015
                  Checkin service    0.028886
                 Baggage handling    0.022399
                 Inflight service    0.019399
                     Seat comfort    0.017230
                      Cleanliness    0.015682
                    Gate location    0.011324
                              Age    0.010642
                  Flight Distance    0.010367
                 Leg room service    0.009078
                        Class_Eco    0.005225
         Arrival Delay in Minutes    0.004319
       Departure Delay in Minutes    0.002640
Departure/Arrival time convenient    0.001871
                   Food and drink    0.001830


In [125]:
engineered_features = ['overall_service', 'entertainment_score', 'total_delay', 
                      'has_delay,delay_ratio','distance_category','age_group','low_ratings_count','high_ratings_count']


for feature in engineered_features:
    if feature in importance_df['Feature'].values:
        importance = importance_df.loc[importance_df['Feature'] == feature, 'Importance'].values[0]
        rank = importance_df[importance_df['Feature'] == feature].index[0] + 1
        print(f"{feature}: Важность = {importance:.4f}, Ранг = {rank}")
    else:
        related_features = [f for f in importance_df['Feature'] if feature in f]
        for related_feature in related_features:
            importance = importance_df.loc[importance_df['Feature'] == related_feature, 'Importance'].values[0]
            rank = importance_df[importance_df['Feature'] == related_feature].index[0] + 1
            print(f"{related_feature}: Важность = {importance:.4f}, Ранг = {rank}")

distance_category_medium: Важность = 0.0001, Ранг = 24
distance_category_short: Важность = 0.0001, Ранг = 25
age_group_young_adult: Важность = 0.0007, Ранг = 29
age_group_child: Важность = 0.0003, Ранг = 26
age_group_senior: Важность = 0.0001, Ранг = 28
age_group_elderly: Важность = 0.0001, Ранг = 27


In [126]:
results = {
    'Модель': ['Логистическая регрессия', 'Логистическая регрессия (SMOTE)', 
               'KNN', 'KNN (SMOTE)', 
               'Дерево решений', 'Дерево решений (SMOTE)'],
    'Accuracy': [accuracy, accuracy_smote, 
                 accuracy_knn, accuracy_knn_smote, 
                 accuracy_tree, accuracy_tree_smote],
    'Precision': [precision, precision_smote, 
                  precision_knn, precision_knn_smote, 
                  precision_tree, precision_tree_smote],
    'Recall': [recall, recall_smote, 
               recall_knn, recall_knn_smote, 
               recall_tree, recall_tree_smote],
    'F1-score': [f1, f1_smote, 
                 f1_knn, f1_knn_smote, 
                 f1_tree, f1_tree_smote]
}


In [127]:
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

                         Модель  Accuracy  Precision   Recall  F1-score
        Логистическая регрессия  0.875806   0.869111 0.839867  0.854238
Логистическая регрессия (SMOTE)  0.868203   0.840840 0.858301  0.849481
                            KNN  0.927437   0.938575 0.890838  0.914084
                    KNN (SMOTE)  0.924502   0.923270 0.900611  0.911799
                 Дерево решений  0.950101   0.946937 0.937368  0.942128
         Дерево решений (SMOTE)  0.948802   0.942889 0.938701  0.940790


In [128]:
best_model_idx = results_df['F1-score'].idxmax()
best_model = results_df.loc[best_model_idx]
print(f"Модель: {best_model['Модель']}")
print(f"F1-score: {best_model['F1-score']:.4f}")
print(f"Accuracy: {best_model['Accuracy']:.4f}")
print(f"Precision: {best_model['Precision']:.4f}")
print(f"Recall: {best_model['Recall']:.4f}")

Модель: Дерево решений
F1-score: 0.9421
Accuracy: 0.9501
Precision: 0.9469
Recall: 0.9374


Проведенное сравнение моделей машинного обучения показало интересные ПРИКОЛЫ.
Логистическая регрессия показала стабильные и предсказуемые результаты вне зависимости от применения техники балансировки классов SMOTE.  Алгоритм K-ближайших соседей также показал качественные результаты, однако его применение растягивается на изучение объёма данных на 5 минут и даже Intel I9 не спасает от турбо надува самолёта.
ДЕРЕВО РЕЩЕНИЙ выделилось как наиболее эффективная модель, особенно при использовании техники SMOTE для балансировки классов.
Показатель F1-score продемонстрировал рост для всех моделей после применения SMOTE, хотя незначительно.
Наиболее значимыми оказались такие характеристики, как возможность онлайн-регистрации на рейс, качество Wi-Fi соединения в полете и комфортность посадочных мест. 
Признаки, созданные в процессе feature engineering полное... показали незначительные результаты.